In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import warnings
import joblib
import plotly.express as px
from google.colab import drive
from datetime import date, timedelta
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
# from catboost import CatBoostClassifier, CatBoostRegressor, Pool
# from plot_metric.functions import BinaryClassification
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import log_loss, roc_auc_score
import seaborn as sns
import plotly.express as px

warnings.filterwarnings('ignore')
pd.options.display.max_columns = 500

from google.colab import auth
import gspread
from google.auth import default
from google.cloud import bigquery
client = bigquery.Client(project='dev-sd-lake')

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
drive.mount('/content/drive',  force_remount=True)

Mounted at /content/drive


In [ ]:
sales_prob_price_model = joblib.load("/content/drive/MyDrive/PSI_Modeling_Data/Fern/sales_prob_price_model.pkl")
domestic_export_price_model = joblib.load("/content/drive/MyDrive/PSI_Modeling_Data/Fern/domestic_export_price_model.pkl")
X_val = joblib.load("/content/drive/MyDrive/PSI_Modeling_Data/Fern/X_val.pkl")

In [ ]:
X_val = X_val.drop('domestic_export', axis = 1)

In [ ]:
def get_dummy_prices(x):
  return [x*0.1, x*0.15, x*0.2, x*0.25, x*0.3, x*0.35, x*0.4, x*0.45, x*0.5, x*0.6, x*0.7, x*0.8]

X_val['new_price_per_unit_usd'] = X_val['retail_price_per_unit_usd'].apply(get_dummy_prices)
X_val = X_val.explode('new_price_per_unit_usd').reset_index(drop = True)
X_val['order_price_per_unit_usd'] = X_val['new_price_per_unit_usd']
X_val = X_val.drop('new_price_per_unit_usd', axis = 1)

In [ ]:
X_val['probs'] = sales_prob_price_model.predict_proba(X_val)[:,1]
X_val['expected_revenue'] = X_val['order_price_per_unit_usd'] * X_val['probs']
X_val['probs'] = X_val['probs'] * 100

In [ ]:
num = np.random.randint(0, 1000)
X_val[12*num:12*num + 12]

,sku_number,brand,product_category,product_subcategory,seller_name,total_units,shelf_life_remaining_days,time,listing_condition,retail_price_per_unit_usd,order_price_per_unit_usd,probs,expected_revenue
3756,68833195,love_beauty_&_planet,hair_care,conditioner,ulid,5124.0,597.0,14.0,excess,4.49,0.449,26.891787,0.120744
3757,68833195,love_beauty_&_planet,hair_care,conditioner,ulid,5124.0,597.0,14.0,excess,4.49,0.6735,60.866734,0.409937
3758,68833195,love_beauty_&_planet,hair_care,conditioner,ulid,5124.0,597.0,14.0,excess,4.49,0.898,77.694002,0.697692
3759,68833195,love_beauty_&_planet,hair_care,conditioner,ulid,5124.0,597.0,14.0,excess,4.49,1.1225,89.513689,1.004791
3760,68833195,love_beauty_&_planet,hair_care,conditioner,ulid,5124.0,597.0,14.0,excess,4.49,1.347,89.227724,1.201897
3761,68833195,love_beauty_&_planet,hair_care,conditioner,ulid,5124.0,597.0,14.0,excess,4.49,1.5715,81.686081,1.283697
3762,68833195,love_beauty_&_planet,hair_care,conditioner,ulid,5124.0,597.0,14.0,excess,4.49,1.796,67.178823,1.206532
3763,68833195,love_beauty_&_planet,hair_care,conditioner,ulid,5124.0,597.0,14.0,excess,4.49,2.0205,70.639417,1.427269
3764,68833195,love_beauty_&_planet,hair_care,conditioner,ulid,5124.0,597.0,14.0,excess,4.49,2.245,74.862149,1.680655
3765,68833195,love_beauty_&_planet,hair_care,conditioner,ulid,5124.0,597.0,14.0,excess,4.49,2.694,57.188551,1.54066


In [ ]:
px.line(X_val[12*num:12*num + 12], x="order_price_per_unit_usd", y="probs", title = 'Selling Price probability v/s Increasing per unit price')

In [ ]:
X_val = joblib.load("/content/drive/MyDrive/PSI_Modeling_Data/Fern/X_val.pkl")

In [ ]:
def get_dummy_prices(x):
  return [x*0.1, x*0.2, x*0.3, x*0.4, x*0.5, x*0.6, x*0.7, x*0.8]

X_val['new_price_per_unit_usd'] = X_val['retail_price_per_unit_usd'].apply(get_dummy_prices)
X_val = X_val.explode('new_price_per_unit_usd').reset_index(drop = True)
X_val['order_price_per_unit_usd'] = X_val['new_price_per_unit_usd']
X_val = X_val.drop('new_price_per_unit_usd', axis = 1)

In [ ]:
X_val['probs'] = domestic_export_price_model.predict_proba(X_val)[:,1]
X_val['expected_revenue'] = X_val['order_price_per_unit_usd'] * X_val['probs']

In [ ]:
X_val.head()

,sku_number,brand,product_category,product_subcategory,seller_name,total_units,shelf_life_remaining_days,time,listing_condition,retail_price_per_unit_usd,order_price_per_unit_usd,domestic_export,probs,expected_revenue
0,PUMAANIMATRIXIIDP(36136404)SLLIPERFORMENDARKSH...,puma,"apparels,_footwear,_accessories",footwear,excess2sell,1.0,NaN,NaN,excess,6.897468,0.689747,domestic,0.673077,0.464253
1,PUMAANIMATRIXIIDP(36136404)SLLIPERFORMENDARKSH...,puma,"apparels,_footwear,_accessories",footwear,excess2sell,1.0,NaN,NaN,excess,6.897468,1.379494,domestic,0.309225,0.426574
2,PUMAANIMATRIXIIDP(36136404)SLLIPERFORMENDARKSH...,puma,"apparels,_footwear,_accessories",footwear,excess2sell,1.0,NaN,NaN,excess,6.897468,2.06924,domestic,0.031545,0.065273
3,PUMAANIMATRIXIIDP(36136404)SLLIPERFORMENDARKSH...,puma,"apparels,_footwear,_accessories",footwear,excess2sell,1.0,NaN,NaN,excess,6.897468,2.758987,domestic,0.007639,0.021076
4,PUMAANIMATRIXIIDP(36136404)SLLIPERFORMENDARKSH...,puma,"apparels,_footwear,_accessories",footwear,excess2sell,1.0,NaN,NaN,excess,6.897468,3.448734,domestic,0.001484,0.005118
